In [2]:
print("Suii")

Suii


In [ ]:
from dotenv import load_dotenv
import os
load_dotenv()

In [ ]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY is missing in your .env")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [ ]:
from langchain_openai import ChatOpenAI

In [ ]:
chat_llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
chat_llm.invoke("Hello, how are you?").content


In [ ]:
from typing_extensions import TypedDict, Annotated
import operator

In [ ]:
from langchain_core.messages import AnyMessage, HumanMessage, AIMessage

In [ ]:
class GraphState(TypedDict):
    messages: Annotated[list[AnyMessage], operator.add]

In [ ]:
{
    "messages": [HumanMessage(content="Hi, this is Sunny. Say hello in detail.")]
}

In [ ]:
["hi"]

In [ ]:
["hi","how are you?"]

In [ ]:
def llm_call(state: GraphState) -> dict:
    """Call the LLM using conversation messages and append AI response."""
    response = chat_llm.invoke(state["messages"])  # AIMessage
    return {
        "messages": [response]
    }


In [ ]:
def token_counter(state: GraphState) -> dict:
    """Count tokens (simple word count) in the last AI message."""
    last_msg = state["messages"][-1]
    text = last_msg.content
    token_number = len(text.split())
    summary = f"Total token number in the generated answer (word count) is {token_number}"
    return {
        "messages": [AIMessage(content=summary)]
    }


In [ ]:
from langgraph.graph import StateGraph

In [ ]:
builder = StateGraph(GraphState)

In [ ]:
builder.add_node("llm_call", llm_call)
builder.add_node("token_counter", token_counter)

In [ ]:
builder.set_entry_point("llm_call")
builder.add_edge("llm_call", "token_counter")
builder.set_finish_point("token_counter")


In [ ]:
app = builder.compile()

In [ ]:
app.get_graph()

Graph(nodes={'__start__': Node(id='__start__', name='__start__', data=RunnableCallable(tags=None, recurse=True, explode_args=False, func_accepts={}), metadata=None), 'llm_call': Node(id='llm_call', name='llm_call', data=llm_call(tags=None, recurse=True, explode_args=False, func_accepts={}), metadata=None), 'token_counter': Node(id='token_counter', name='token_counter', data=token_counter(tags=None, recurse=True, explode_args=False, func_accepts={}), metadata=None), '__end__': Node(id='__end__', name='__end__', data=None, metadata=None)}, edges=[Edge(source='__start__', target='llm_call', data=None, conditional=False), Edge(source='llm_call', target='token_counter', data=None, conditional=False), Edge(source='token_counter', target='__end__', data=None, conditional=False)])

In [ ]:
from IPython.display import Image, display

In [ ]:
display(Image(app.get_graph().draw_mermaid_png()))

In [ ]:
result = app.invoke({
    "messages": [HumanMessage(content="Hi, this is Sunny. Say hello in detail.")]
})

In [ ]:
result

In [ ]:
for m in result["messages"]:
    print(type(m).__name__, ":", m.content)